In [ ]:
# Importing Libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import argparse, copy

from model import *
from utils import *
from measures import *

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device))

from google.colab import drive
drive.mount('/content/drive')


# pip install -e .

In [ ]:
batch_size = 64
test_batch_size = 1000
epochs = 2
lr = 1e-3
gamma = 0.7
seed = 12345
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_cuda = torch.cuda.is_available()
train_loader, test_loader = dataloaders(batch_size, use_cuda)
print (len(train_loader))
# for i in train_loader:
#     x = i[0].view(64, -1, 28*28)
#     print (x.size())
#     break
model_init = Net().to(device)
# CHANGE AS APPROPRIATE
optimizer = optim.SGD(model_init.parameters(), lr=lr)
# optimizer = optim.LBFGS(model.parameters(), lr=lr)
# scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
# saving checkpoints during/after training;;;can save other variables as well
# checkpoint = {'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
train_losses, test_losses = [], []

In [ ]:
# test accuracy problem.....
def main():
    train_size = 60032
    sigmas = [1.0,2.0,3.0,4.0] # 1e-3, 1e-2, 1e-1, 1e-0
    delta = 1e-2
    mean = 0.0
    l_weight_norms = []
    l_sharpness = []
    var_gradients = []  # maybe with respect to batch size.....
    """run for each sigma"""
    for sigma in sigmas:
        grads = []
#         resetting model everytime
        model = copy.deepcopy(model_init)
        for epoch in range(1, epochs + 1):
            train_loss = train(model, device, train_loader, optimizer, epoch, batch_size)
            train_losses.append(train_loss)
    #             save_checkpoint(checkpoint) #Change file path as appropriate
            test_loss = test(model, device, test_loader, batch_size)
            test_losses.append(test_loss)
            # scheduler.step()
            gradients = get_gradients(model)  # a list of values
            grads.append(gradients) 
        variance_score = compute_grad_variance(grads)
        var_gradients.append(variance_score)

        weight_l2_norm = cal_l2_norm(model)
        l_weight_norms.append(weight_l2_norm)
        p_model = weight_pertubation(model, mean, sigma, device)
        p_test_error = test(p_model, device, test_loader, batch_size)
        sharpness = compute_sharpness(model, p_test_error, train_loss, weight_l2_norm, train_size, sigma, delta)
        l_sharpness.append(sharpness)
        show_losses(train_losses, test_losses)  
    show_sharpness_norm(l_sharpness, l_weight_norms, sigmas)
        
if __name__ == '__main__':
    main()        